# Chapter 1. Introduction to DataScience

### < Themes > 
* 1). Finding Key Connectors.
* 2). Data Scientist You May Know.

In [141]:
# at this stage in the book we haven't actually installed matplotlib,
# comment this out if you need to
from matplotlib import pyplot as plt

##########################
#                        #
# FINDING KEY CONNECTORS #
#                        #
##########################

users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" },
    { "id": 10, "name": "Jen" }
]

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]


# first give each user an empty list
for user in users:
    user["friends"] = []

# and then populate the lists with friendships
for i, j in friendships:
    # this works because users[i] is the user whose id is i
    users[i]["friends"].append(users[j]) # add i as a friend of j
    users[j]["friends"].append(users[i]) # add j as a friend of i

def number_of_friends(user):
    """how many friends does _user_ have?"""
    return len(user["friends"]) # length of friend_ids list

total_connections = sum(number_of_friends(user)
                        for user in users) # 24

num_users = len(users)
avg_connections = total_connections / num_users # 2.4

In [142]:
# Data Scientist You May Know : (Suggester in SNS)

def FOF_id_list(user_id):
    
    # 1. query input id and find the corresponding dictionary
    for user in users:
        if user['id'] == user_id:
            the_user_dict = user
    
    # 2. make an empty list to return result.
    fof_list = []
    
    # 3. iterate friends of input and append the friends of the ones who are the input's friends.
    for friend in the_user_dict['friends']:
        for friend in friend['friends']:
            if friend['id'] != user_id:
                fof_list.append(friend['id'])
    
    fof_list = list(set(fof_list))
    
    return fof_list

In [143]:
# In Python, try harder to do it in list comprehension.

def FOF_id_list_2(user_id):
    for user in users:
        if user['id'] == user_id:
            the_user_dict = user
    
    return list(
           set(
            [ foaf['id']
            for friend in the_user_dict['friends']
            for foaf in friend['friends'] 
            if foaf['id']!=user_id ]
              )
               )

In [144]:
FOF_id_list_2(0)

[1, 2, 3]

#  Exclude Mutual Friends in Suggester System.

In [145]:
# import counter func
from collections import Counter

# make not_the_same checking function
def not_the_same(user, other_user):
    return user['id'] != other_user['id']

# make not_friends function
def not_friends(user, other_user):
    return all( not_the_same(friend, other_user) for friend in user['friends'] )

# make foaf not same , not mutual suggest list counter

def friends_of_friend_ids(user):
    return Counter(
    foaf['id']
    for friend in user['friends']
    for foaf in friend['friends']
    if not_the_same(user, foaf) and not_friends(user, foaf)
    )

In [146]:
customer_foaf = dict()
for i in range(len(users)):
    key = i
    value = friends_of_friend_ids(users[i]) #foaf 인 경우가 많을 수록 추천해줬을 때 친할 확률이 더 크겠다.
    customer_foaf[key] = value

customer_foaf

{0: Counter({3: 2}),
 1: Counter({4: 1}),
 2: Counter({4: 1}),
 3: Counter({0: 2, 5: 1}),
 4: Counter({1: 1, 2: 1, 6: 1, 7: 1}),
 5: Counter({3: 1, 8: 2}),
 6: Counter({4: 1, 7: 2, 9: 1}),
 7: Counter({4: 1, 6: 2, 9: 1}),
 8: Counter({5: 2}),
 9: Counter({6: 1, 7: 1}),
 10: Counter()}

# Send suggestion of 'FriendsOfAFriend' that have the most counts : which supposedly are having the highest probabilty to be the user's friend >

In [147]:
# recommend friends who has max foaf connection
for i in range(len(customer_foaf)):
    
    # get counter list 
    tuple_list = customer_foaf[i].most_common()
    
    # query the tuples that have max_count and choose them out
    max_count_ids = [x for x,y in tuple_list if y == (max([b for a,b in tuple_list]))]
    
    # suggest those Xs to a user
    if len(max_count_ids)!=0:
        print('suggested user_id {} to user_id {} '.format(max_count_ids, i))
    else :
        print('None suggested to user_id {}'.format(i))

suggested user_id [3] to user_id 0 
suggested user_id [4] to user_id 1 
suggested user_id [4] to user_id 2 
suggested user_id [0] to user_id 3 
suggested user_id [1, 2, 6, 7] to user_id 4 
suggested user_id [8] to user_id 5 
suggested user_id [7] to user_id 6 
suggested user_id [6] to user_id 7 
suggested user_id [5] to user_id 8 
suggested user_id [6, 7] to user_id 9 
None suggested to user_id 10


## Another way to choose out the keys with the biggest value : operator module

In [148]:
# in dealing with Counter and getting the the key of maximum value
# I could also use 'operator'

import operator
stats = {'a':3000, 'b':3000, 'c': 300, 'd': 400}
max(stats.iteritems(), key=operator.itemgetter(0))
min(stats.iteritems(), key=operator.itemgetter(1))

('c', 300)

# Friend Suggestion by shared interests

In [149]:
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

In [150]:
# import module
from collections import defaultdict
